# Configuration
Here, we define the configuration of our segmentation pipeline.

In [1]:
### Michael Engel ### 2022-04-25 ### main.ipynb ###
### adapted by Niklas Eisl, Colin Moldenhauer, 2022/23 ###
import torch.cuda

from libs.ConfigME import Config
import os
import platform
import datetime as dt
from sentinelhub import SHConfig

Incorporating libs!


Now, we can initialize the configuration file with a proper name and identifiers for storing.

In [2]:
config = Config(
    name = 'Treecurrent', # name of the project
    savename = None, # basic name to store stuff
    savename_config = None # name of configuration file
)

Our pipeline is defined by 3 notebooks.

In [3]:
config.file_DataAcquisition = "01_DataAcquisition.ipynb"
config.file_DataNormalisation = "02_DataNormalisation.ipynb"
config.file_TrainingValidationTesting = "03_TrainingValidation.ipynb"

Let's define the directories we are working with, i.e. in which directories to store our `EOPatches` and results.
By that, we ensure that everything is only defined once.

In [4]:
config.dir_input = os.path.join(os.getcwd(),"inputs")

In [5]:
config.basedir = os.getcwd()
config.dir_results = os.path.join(config["basedir"], "results", config["savename"])
config.dir_checkpoints = os.path.join(config["dir_results"], "checkpoints")
config.dir_tensorboard = os.path.join(config["dir_results"], "tensorboard")
config.dir_imgs = os.path.join(config["dir_results"], "imgs")
config.dir_imgs_validation = os.path.join(config["dir_imgs"], "PredictionValidation")
config.dir_imgs_test = os.path.join(config["dir_imgs"], "PredictionTest")

In [6]:
config.dir_data = os.path.join(config["basedir"], "data")
config.dir_train = os.path.join(config["dir_data"], "train")
config.dir_val = os.path.join(config["dir_data"], "val")
config.dir_test = os.path.join(config["dir_data"], "test")

config.dir_cache = os.path.join(os.getcwd(),"cache")

Let's load our **credentials** for Sentinel Hub from storage. Make sure you have saved your credentials beforehand.

In [7]:
config.SHconfig = SHConfig()
if not config["SHconfig"].sh_client_id or not config["SHconfig"].sh_client_secret:
    print("Warning! To use Process API, please provide the credentials (OAuth client ID and client secret).")

others

In [8]:
config.AOI = os.path.join(config["dir_input"], "bavaria_south.json")
config.split_percentages = [.7, .1, .2]       # train, val, test percentages of patches
config.split_direction = "v"                # split the AOI from west (train) to east (test)
config.patchpixelwidth = 256

config.maxcc = 1            # maximum cloud coverage for training samples
config.maxcc_ref = 0.3      # maximum cloud coverage for reference samples
config.resolution = 10      # pixel ground resolution

config.feature_name = "data"                                # feature name for data
config.bands = ["B02", "B03", "B04", "B08", "B8A", "B11"]   # required bands for SentinelHub

config.time_difference = dt.timedelta(weeks=2)              # minimum time difference between training samples of time series
config.time_difference_mosaic = dt.timedelta(days=1)        # mosaicking time difference
config.time_diff_ref = dt.timedelta(days=2)                 # minimum time difference between references (choose low for good coverage over the year)
config.time_interval = ("2021-01-01", "2022-01-01")         # time interval to pick reference timestamps
config.num_observations = 8                                 # time series length (excluding reference)

config.threads = 1 if platform.system()=="Windows" else 12

general ML parameters

In [9]:
config.n_epochs = 10
config.batch_size = 6
config.checkpoint_bestloss = True
config.checkpoint_freq = 2
config.device = "cuda" if torch.cuda.is_available() else "cpu"
config.eval_freq = 1
config.seed = 42

model specific ML parameters

In [10]:
config.model_savename = config["savename"]
config.model_savename_bestloss = config["model_savename"]+"_bestloss"
config.model_savename_inference = config["savename"]+"_inference"
config.model_savename_inference_bestloss = config["model_savename_inference"]+"_bestloss"
config.module_model = "models.convlstm.ConvLSTM"
config.kwargs_model = {
    "input_dim": len(config.bands),
    "hidden_dim": [20, 20, 1],
    "kernel_size": (3, 3),
    "num_layers": 3,
    "reorder_dims": (0, 2, 1, 3, 4), # -> (b, t, c, h, w) dataloader: b c t w h
    "bias": True,
    "return_all_layers": False
}

Here, we will use the [MSELoss](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss).
We will not apply reduction since we would like to apply our mask manually.

In [11]:
config.module_loss = "utils.loss.MSELossMasked"
config.kwargs_loss = {
    "size_average": None,
    "reduce": None,
    "reduction": "mean"
}

We will use the standard [Adam Optimizer](https://pytorch.org/docs/stable/generated/torch.optim.Adam.html).

In [12]:
config.module_optimizer = "torch.optim.Adam"
config.kwargs_optimizer = {
    "lr": 0.00007,
    "betas": (0.9, 0.999),
    "eps": 1e-08,
    "weight_decay": 1e-06,
    "amsgrad": False
}

For evaluation, we need some metrics, for example the MSE.

In [13]:
config.module_metric = ["utils.metrics.masked_mse_loss", "utils.metrics.squared_variance_error"]

For the data normalisation, we have

In [14]:
config.savename_tdigest = config["savename"]+"_TDigest.npy" 
config.savename_scaler = config["savename"]+"_QuantileScaler.dill" 

# quantiles
config.scaler_maxquantile = 0.99
config.scaler_minquantile = 0.01
# special values
config.scaler_nanval = [0] * len(config.bands)
config.scaler_infval = [0] * len(config.bands)
# scale to following interval
config.scaler_valmin = 0
config.scaler_valmax = 1

We may not forget to store our configuration file to disk.

In [ ]:
config.checkdir()

In [ ]:
config.checkfile()

In [ ]:
config.checkmodule()

In [18]:
file = config.save()
file2 = config.save(os.path.join(config["dir_results"],config["savename_config"])) # saving to results folder

In [ ]:
config.print()